### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2018M04D19', 'UTC 13:38')


# Settings

In [2]:
MAXTASKS = 10
CANCELTASKS = 0

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9


In [8]:
detailedTasks

,creation_timestamp_ms,description,id,state,task_type,update_timestamp_ms
0,1524145213911,global_historical_PDomWN_month_m_5min_1960_201...,JJZGWFKGLHFENXWH37YGXAEI,READY,EXPORT_IMAGE,1524145213911
1,1524145212742,global_historical_PDomWN_month_m_5min_1960_201...,MVVWDXFYHHBBJHCO7UWVV5OK,READY,EXPORT_IMAGE,1524145212742
2,1524145210884,global_historical_PDomWN_month_m_5min_1960_201...,XTVFJUQF4BMHVYVCA2ADAIFI,READY,EXPORT_IMAGE,1524145210884
3,1524145209661,global_historical_PDomWN_month_m_5min_1960_201...,TZYHR2BYKLYQRLY6FRMCW4O4,READY,EXPORT_IMAGE,1524145209661
4,1524145207948,global_historical_PDomWN_month_m_5min_1960_201...,ID3UTMKU4KHVU6SOHHUUDFBP,READY,EXPORT_IMAGE,1524145207948
5,1524145206155,global_historical_PDomWN_month_m_5min_1960_201...,EYYLYD7N43QVX7VJMROMI54T,READY,EXPORT_IMAGE,1524145206155
6,1524145204433,global_historical_PDomWN_month_m_5min_1960_201...,7KYXXF2HKJPOHXFT5HZD3HBH,READY,EXPORT_IMAGE,1524145204433
7,1524145202641,global_historical_PDomWN_month_m_5min_1960_201...,R5G7FR4P5F5ZGQSUUKVFAWJI,READY,EXPORT_IMAGE,1524145202641
8,1524145200956,global_historical_PDomWN_month_m_5min_1960_201...,7EMFEHNWT5EKUQ57CKU5AFUT,READY,EXPORT_IMAGE,1524145200956
9,1524145199102,global_historical_PDomWN_month_m_5min_1960_201...,TJJCXV6TMW2XHZH7FNEZV6RV,READY,EXPORT_IMAGE,1524145199102


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)